# Analise de Indicadores Financeiros e Fundos de Investimentos


# 1 - Introdução

Esse caderno tem como objetivo exemplificar como acessar as principais fontes de informações sobre indicadores financeiros da economia brasileira, além dar uma breve introdução sobre cada um deles.

Ao fim desse caderno você será capaz de:
- Entender quais são os pricipais indicadores econômicos brasileiros e para que eles servem.
- Determinar quais são as fontes oficiais desses indicadores, como acessar essas fontes e extrair os dados necessários para suas análises.
- Manipular esses dados através da biblioteca Pandas e construir visualizações gráficas sobre esses dados através da biblioteca ploty.

## 1.2 - Índice Bovesp

## 1.3 - ICPA+6

## 1.4 - Dolar

As infomrações da contação oficial do Dollar pode ser obitida através da API do portal de dados abertos do Banco Central através na URL <https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/swagger-ui3#/>.

## 1.5 - Ouro

## 1.6 - S&P 500

## 1.7 - IFIX

# 2 - Code Session

## 2.1 - Bibliotecas requeridas

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

## 2.2 - Código para acesso ao sistema gerenciador de séries do Banco Central.

In [ ]:
# Código das séries históricas no SGS do BC.
SGS_SELIC = 11
SGS_CDI   = 12
SGS_IPCA  = 433 

In [ ]:
# Recupera dados de série histórica do banco central, atrávés do código da série.
def get_sgs_data(serie):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(serie)
  df = pd.read_json(url)
  return df

## 2.3 - Código para acesso ao sistema gerenciador de séries da Comissão de Valores Mobiliários.

In [ ]:
# Recupera os dados de cadastro dos fundo de investimento da CVM
def get_cvm_fi_data():
  url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'
  return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

In [ ]:
# Recupera as informações financeiras dos fundos de invetimentos da CMV em um determinado período de tempo.
def get_cvm_fi_info(dt_inicio, dt_fim):
  datas = pd.date_range(dt_inicio, dt_fim, freq='MS') 
  fi_info = pd.DataFrame()

  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv'.format(data.year, data.month)
      fi_info_month = pd.read_csv(url, sep=';')    
    
    except: 
      print("File {} not found".format(url))    

    fi_info = pd.concat([fi_info, fi_info_month], ignore_index=True)

  return fi_info

In [ ]:
# Consulta os informes rendimento para um fundo epecifico.
def consulta_fundo(informes, cnpj):  
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0]
  return fundo

# 3 -  Data Section

## 3.1 - Indices comparativos de performace econômica

### CDI

> O **CDI** 

O **Certificado de Depósito Interbancário** são títulos emitidos por instituições financeiras, negociados apenas entre os bancos, que regulam empréstimos de curtíssimo prazo (geralmente um dia) entre os bancos. O CDI tem a finalidade de regular o fluxo de caixa nas instituiçõe bancárias e serve como referência de juros, avaliação de investimentos, além de expressar a expectativa das instituições financeiras frente ao mercado. A série histórica do índice é calculada pela CTIP que é uma empresa particular responsável pela integração do mercado financeiro e faz parte da B3.

Os dados do CDI podem ser acessados através a partir do site da [CETIP][1], da [API][2] do portal de dado abertos Banco Central do Brasil ou do [sistema gestor de séries][3] do BCB. Contudo existe uma difença entre as informações providas entre a CETIP e o Banco Central.

Para as séries extraídas da CETIP temos como indicador o **Fator Diário** que é nesse caso é um número decimal que representa o rendimento anual do CDI, já no BC esse valor é percentual. Nas séries abaixo, vamos trabalhar com o indice reportado pelo BCB uma vez que ela é a mais usada.

> O ** CDI ** 

[1]: http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v05_template_informacoes_di.asp?str_Modulo=completo&int_Idioma=1&int_Titulo=6&int_NivelBD=2
[2]: https://dadosabertos.bcb.gov.br/
[3]: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

In [ ]:
# Recupera a série histórica do CDI diário do BCB.
df_cdi = get_sgs_data(SGS_CDI)
df_cdi['data'] = pd.to_datetime(df_cdi['data'], dayfirst= True)
df_cdi.set_index('data', inplace= True)

# Recalcula o valor do CDI mensal
df_cdi_mes = df_cdi.resample('M').sum()

# Calcula o CDI acumulado através dos anos.
df_cdi_acumulado = ((1 + df_cdi/100).cumprod() - 1) * 100

df_cdi_temp = df_cdi.copy()
df_cdi_temp['year'] = df_cdi_temp.index.year
df_cdi_temp['doy']  = df_cdi_temp.index.dayofyear
df_cdi_porano = df_cdi_temp.pivot(index='doy',columns='year',values='valor')
df_cdi_porano = df_cdi_porano.fillna(method='bfill')


Amostra dos dados da série histórica do CDI do Banco Central do Brasil, com a linha dos seu valor dia a dia através dos anos.

In [ ]:
df_cdi.head()

In [ ]:
px.line(df_cdi)

A mesma série com os valores consolidados por mês.

In [ ]:
df_cdi_mes.head()

In [ ]:
px.line(df_cdi_mes)

In [ ]:
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=df_cdi.index, y=df_cdi['valor'], name='CDI Diario'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_cdi_mes.index,y=df_cdi_mes['valor'],name='CDI Mensal'),row=2,col=1)
fig.update_layout(height=768, width=1024, title_text="Taxa CDI")
fig.show()

A amostra a seguir mostra o CDI acumulado através dos anos.

In [ ]:
df_cdi_acumulado.head()

In [ ]:
px.line(df_cdi_acumulado)

A série a seguir, mostra o comportamento do CDI ano a ano, onde cada uma das linhas apresentadas representa o valor CDI naquele dia do ano.

In [ ]:
df_cdi_porano.head()

In [ ]:
px.line(df_cdi_porano)

### SELIC

> A **SELIC** representa a taxa média de juros do empréstimos feitos entre bancos que tem títulos públicos federais como garantia, que é registrado no Sistema Especial de Liquidação e Custódia e serve como base para a politica monetária e possibilitar a estabilidade economica.

A taxa SELIC repesenta o juros básicos da economia brasileira. SELIC significa **Sistema Especial de Liquidação e Custódia** e é um sistema controlado pelo Banco Centra do Brasil e serve para controlar a quantidade de dinheiro em circulação através da regulação do crédido concedido aos consumidores. A SELIC é um dos mecânismos utlizados pelo governo para controlar a inflação. Quanto maior a SELIC mais altos ficam os juros para o consumidor deixando o crédito mais caro desistimulando assim o consumo de modo a frear a economia e diminuir o ritimo da inflação, sendo o contrário verdadeiro também, quanto menor a SELIC mais barato fica o crédito estimulando o consumo e acelerando a inflação. A taxa SELIC serve de base para o CDI e é normal que ambas estejam sempre no mesmo valor.
 

In [ ]:
#Recupera a série histórica da SELIC do sistema do Banco Central do Brasil.
df_selic = get_sgs_data(SGS_SELIC)
df_selic['data'] = pd.to_datetime(df_selic['data'], dayfirst= True)
df_selic.set_index('data', inplace= True)
df_selic.head()

In [ ]:
px.line(df_selic)

O CDI tem como base a SELIC então é normal que ambos sigam com o mesmo valor. Mas diferença entre elas podem ocorrer quando 

In [ ]:
#Traça os gráficos comparativos entre o CDI e a SELIC
fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(x=df_cdi.index, y=df_cdi['valor'], name='CDI'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_selic.index,y=df_selic['valor'],name='Taxa SELIC'),row=2,col=1)
fig.update_layout(height=768, width=1024, title_text="Comparativo CDI x SELIC")
fig.show()

### IPCA

In [ ]:
# Recupera o IPCA do sistema gestor de séries do BCB.
df_ipca = get_sgs_data(SGS_IPCA)
df_ipca['data'] = pd.to_datetime(df_ipca['data'], dayfirst= True)
df_ipca.set_index('data', inplace= True)

In [ ]:
px.line(df_ipca)

### Poupança

## 3.2 Analise sobre os indices de performance econômica.

### Pontos de análise de interesse.

#### Governos da Nova República

In [ ]:
JOSE_SARNEY       = ('1985-03-15','1990-03-15')
FERNANDO_COLLOR   = ('1990-03-15','1992-12-29')
ITAMAR_FRANCO     = ('1992-12-29','1995-01-01')
FERNANDO_HENRIQUE = ('1995-01-01','2003-01-01')
LULA1             = ('2003-01-01','2011-01-01')
LULA2                                  = ()
DILMA             = ('2011-01-01','2016-08-31')
TEMER             = ('2016-08-31','2019-01-01')
BOLSONARO         = ('2019-01-01','2023-01-01')

#### Períodos Eleitorais Federal e Estadual.

In [ ]:
# Fernando Herique Cardoso - Primeiro Mandato
ELEICAO_1994_T1 = ('1994-10-03')
ELEICAO_1994_T2 = ('1994-11-15')

# Fernando Herique Cardoso - Segundo Mandato
ELEICAO_1998_T1 = ('1998-10-04')
ELEICAO_1998_T2 = ('1998-11-25')

# Lula - Primeiro Mandato
ELEICAO_2002_T1 = ('2002-10-06')
ELEICAO_2002_T2 = ('2002-10-27')

# Lula - Segundo Mandato
ELEICAO_2006_T1 = ('2006-10-01')
ELEICAO_2006_T2 = ('2006-10-29')

# Dilma - Primeiro Mandato
ELEICAO_2010_T1 = ('2010-10-01')
ELEICAO_2010_T2 = ('2010-10-29')

# Dilma - Segundo Mandato
ELEICAO_2014_T1 = ('2014-10-05')
ELEICAO_2014_T2 = ('2014-10-26')

# Bolsonaro - Primeiro Mandato
ELEICAO_2018_T1 = ('2018-10-07')
ELEICAO_2018_T2 = ('2018-10-28')

#### Crises Econômicas Mundiais

#### Analise do IPCA no Governo Bolsonaro.

In [ ]:
# Recupera o IPCA para o periodo do governo Bolsonaro
dt_inicio, dt_fim = BOLSONARO
df_ipca_bolsonaro = df_ipca.loc[dt_inicio : dt_fim]

#### Analise IPCA no governo Lula

In [ ]:
dt_inicio, dt_fim = LULA

df_ipca = get_sgs_data(SGS_IPCA)
df_ipca['data'] = pd.to_datetime(df_ipca['data'], dayfirst= True)
df_ipca.set_index('data', inplace= True)
df_ipca = df_ipca.loc[dt_inicio : dt_fim]
result = seasonal_decompose(df_ipca)

In [ ]:
fig, (ax1,ax2,ax3, ax4) = plt.subplots(4,1, figsize=(15,16))

result.observed.plot(ax=ax1, title = 'ICPA')
result.trend.plot(ax=ax2, title = 'ICPA Trend')
result.seasonal.plot(ax=ax3, title = 'ICPA Seazonal')
result.resid.plot(ax=ax4, title = 'ICPA Residual')

plt.tight_layout()

In [ ]:
result.seasonal

### Dados sobre os fundos de investimentos


In [ ]:
#Recupera os dados de cadastro dos fundos de investimento da CVM
fundos_cvm = get_cvm_fi_data()
fundos_cvm

Os códigos a seguir são para acesso direto aos indicadores.

In [ ]:
import requests

In [ ]:
# Recupera a Taxa DI com acesso direto a base da CETIP.

url = 'http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v04_10_03_consulta.asp'

req_headers = {
    'Host'                      : 'estatisticas.cetip.com.br',
    'User-Agent'                : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Accept'                    : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language'           : 'en-US,en;q=0.5',
    'Accept-Encoding'           : 'gzip, deflate',
    'Content-Type'              : 'application/x-www-form-urlencoded',
    'Origin'                    : 'http://estatisticas.cetip.com.br',
    'DNT'                       : '1',
    'Referer'                   : 'http://estatisticas.cetip.com.br/astec/series_v05/paginas/lum_web_v04_10_02_gerador_sql.asp',
    'Upgrade-Insecure-Requests' : '1',
    'Sec-GPC'                   : '1'
}

parameters = {
    'str_NomeArquivo'                 : 'WEB_00_DI_Taxas_Over',
    'str_NomeTabela'                  : 'WEB_DI_Taxas_Over',
    'str_Ativo'                       : 'DI',
    'str_ModeloDados'                 : 'TAX_001di',
    'str_Descricao'                   : 'TTTTTT',
    'str_NrLeilao'                    : '_Geral',
    'str_ModeloLeilao'                : '_Geral',
    'str_Descricao_1'                 : '',
    'str_Descricao_2'                 : '',
    'str_Descricao_3'                 : '',
    'chk_Descricao_1'                 : '',
    'chk_Descricao_2'                 : '',
    'chk_Descricao_3'                 : '',
    'bln_MostrarContraparte'          : 'False',
    'str_Populacao'                   : '_Geral',
    'str_FaixaPrazo'                  : '_Geral',
    'str_FaixaPrazoTotalizado'        : '0',
    'str_TipoFaixaPrazo'              : '0',
    'str_TipoEmissao'                 : '0',
    'str_Emissao'                     : '_Geral',
    'str_TipoMoeda'                   : '',
    'str_Moeda'                       : '',
    'str_Observacao'                  : '!DI-CETIP (over)||Observações a respeito de mudança de moeda / volume:|(1) Anos 1986, 1987 e 1988, volume em Cz$ milhões (Cruzado).|(2) A partir de 16/01/1989, corte de três zeros no Cruzado, passando a Cruzado Novo. Volume em NCz$ milhões.|(3) A partir de 15/03/1990, restabelecido o Cruzeiro (Cr$) como moeda nacional. Volume em Cr$ milhões.|(4) A partir de 01/04/1992, em virtude do crescente volume, o mesmo passa a ser divulgado em Cr$ bilhões.|(5) Instituído em 02/08/1993, o Cruzeiro Real (CR$) , com corte de três zeros do Cruzeiro; volume em CR$ milhões.|(6) A partir de 01/11/1993, em virtude do crescimento do volume, o mesmo passa a ser divulgado em CR$ bilhões.|(7) Em Julho/1994, com a implantação do Plano Real, com divisão de valores financeiros por 2.750,00 passando o volume financeiro a ser registrado em milhares de Reais.|           |Observações a respeito da formulação da taxa:|(a) Até 30/06/1989, as taxas dos dias que precediam a fins de semana e feriados eram divididas pelo número de dias destes, de forma a mostrar a taxa over paga pelos dias não úteis.|(b) Até 31/05/1990, taxas divulgadas ao ano de 360 dias, com expressão linear.| Entre 01/06/1990 e 31/12/1997, somente taxas diárias expressas linear ao mês.|(c) A partir de 01/01/1998, taxas médias diárias de DI-Over e de SELIC divulgadas ao ano de 252 dias úteis, com expressão exponencial.|     |Observações a respeito da amostra / base de cálculo da taxa:|(I) Até 29/04/1988, somente taxas envolvendo todas as operações de DI Over.|(II) De 02/05/1988 a 31/05/1990, divisão em Extra-Grupo, Intra-Grupo e Total, sendo o acumulado mensal apenas do Extra-Grupo.|(III) A partir de 01/06/1990, somente taxas do Extra-Grupo.|(IV) A partir de 02/01/1991, inclusão de taxas mínimas, máximas e modais, além do desvio padrão.|     |Observações localizadas:|(i) De acordo com a lei nº 7.320 de 11/06/1985, revogada posteriormente pela lei nº 8.087 de 29/10/1990, foram comemorados por antecipação, nas segundas-feiras, os feriados que caíram nos demais dias da semana, com exceção dos que ocorreram nos sábados e domingos e os dos dias 1º de janeiro (Confraternização Universal), 7 de setembro (Dia da Independência), 25 de dezembro (Natal) e Sexta-Feira Santa.|Com base no exposto, durante o período citado pode ocorrer  a inexistência de taxa para dias considerados como úteis (caso de segundas-feiras imediatamente anteriores a feriados, para onde os mesmos foram deslocados) assim como a existência de taxa para dias considerados como não úteis (caso dos feriados ocorridos entre segunda e sexta-feira, com exceção dos casos já citados).|(ii) Em 23/12/1987, houve também operações diretamente para o dia 28, com cinco dias e um over (saque), com taxa média de 57,66% a.a.|Em 30/12/1987, houve também operações diretamente para o dia 04/01/1988, com cinco dias e dois over (saques), com taxa média de 94,91 % a.a., referentes aos dias 30 e 31/12/1987, abandonando-se, neste caso, a taxa média do dia 31 (60,08 % a.a.).|(iii) Em 12/02/1988, houve também operações diretamente para o dia 18, com seis dias e dois over (saques), com taxa média de 115,72 % a.a.| ||( - ) Dados Insuficientes.',
    'str_NomeAtivoCabecalho'          : 'DI - Depósito Interfinanceiro',
    'str_NomeTipoInformacaoCabecalho' : 'Taxas - DI PRÉ - Over',
    'str_TipoDescricao'               : '5',
    'str_ApresentarTipoOp'            : '0',
    'dta_DataInicio'                  : '01/02/2022',
    'dta_DataFinal'                   : '28/02/2022',
    'str_SQL'                         : 'joao Fdta_DataDivulg, Fdpl_NumOp, Fdpl_Volume, Fsmp_Media, Fsmp_FatorDiario, Fsmp_Minima, Fsmp_Maxima, Fsmp_Moda, Fsmp_DsvPdr, Fsmp_SELIC maria WEB_DI_Taxas_Over Where ( Fdta_DataDivulg >= #02/01/2022# AND Fdta_DataDivulg <= #02/28/2022# ) ORDER BY Fdta_DataDivulg',
    'int_Idioma'                      : '1'
}


cookies = {
    'ASPSESSIONIDCSQRRCAT' : 'GHOGEBCAJDLJDIIFBPJEEPIC',
    'TS01871345' : '01ee5ef46844a27d65554ea6b86003bd46bb7af09e250bf052bf7dc95270e60e93ef143cec76e579ea145d527666e2faba3cc50b73'
}

resp = requests.post(url, data=parameters, headers=req_headers)

In [ ]:
resp.text

In [ ]:
## Recupera a cotação do dolar para um determinado período de tempo.

url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?%40dataInicial='03-01-2022'&%40dataFinalCotacao='03-31-2022'&%24format=text%2Fcsv"

req_headers = {
    "accept" : "application/json;odata.metadata=minimal"
}

resp = requests.post(url, headers= req_headers)
